This notebook is built to develop and train the RCCN model for the 20 m bands (NIR2, RE bands, and SWIR bands). Only blocks 2 and 3 will need to be edited by the user to provide the necessary inputs.

In [1]:
import os
import numpy as np
import rasterio
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,MaxPool2D ,Convolution2D , Add, Dense , AveragePooling2D , UpSampling2D , Reshape , Flatten , Subtract , Concatenate, Cropping2D, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import backend as k
import tensorflow as tf
from tensorflow.keras.utils import plot_model
import tensorflow.keras.utils as ku

from sklearn.model_selection import train_test_split

%load_ext tensorboard

import shutil

2021-12-14 12:46:37.449802: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Change the directory to the location of the outputs by Pyrite (S2 imagery) and BeautySchoolDropout (PS imagery)
os.chdir('...')

# Provide a folder with which to place the final model. 
# IMPORTANT: DO NOT put the same folder as the imagery.Temporary models will be stored there,
# but then deleted. If you do not give a separate location for the complete models, they will also be deleted.
# Ensure there is a '/' at the end.
output_folder = ".../"

In [3]:
# This date (or file identifier) should match what was used in BeautySchoolDropout
date = '17Jun21_'

# These two files correspond to the Pyrite output.  Choose the files with the EXACT SAME ratio (10to80, 20to160)
s210m_80_fn = "T16TCR_20210617T164839_10to80_stack_norm.tif" 
s220m_160_fn = "T16TCR_20210617T164839_20to160_stack_norm.tif" 

# Ground truth. Choose the file that ends with "20_stack_norm.tif"
s220m_20_fn = "T16TCR_20210617T164839_20_stack_norm.tif" 

# If you gave the same identifier above as used in BeautySchoolDropout, this should run without further change
dove20_fn = "%sDove_10m_mosaic.tif" %date # 4 bands + mask
dove20orb_fn = "%sDove_Orbits_10m.tif" %date # orbit(strip number)

s210m_80 = rasterio.open(s210m_80_fn).read().T
s220m_160 = rasterio.open(s220m_160_fn).read().T
dove20 = rasterio.open(dove20_fn).read().T
dove20_orbits = rasterio.open(dove20orb_fn).read().T

print('Shape of downsampled Sentinel-2 10m array: ', s210m_80.shape)
print('Shape of downsampled Sentinel-2 20m array: ', s220m_160.shape)
print('Shape of downsampled Dove array: ', dove20.shape)
print('Shape of downsampled Dove orbit array: ', dove20_orbits.shape)

Shape of downsampled Sentinel-2 10m array:  (500, 500, 5)
Shape of downsampled Sentinel-2 20m array:  (250, 250, 7)
Shape of downsampled Dove array:  (2000, 2000, 5)
Shape of downsampled Dove orbit array:  (2000, 2000, 1)


In [4]:
s220m_20 = rasterio.open(s220m_20_fn).read().T
s220m_20 = s220m_20[:,:, :-1].copy()  # Remove mask for model validation 

s220m_20.shape

(2001, 2001, 6)

In [5]:
encoded = ku.to_categorical(dove20_orbits, dtype='uint16')
print(encoded.shape)

dove20_with_orb = np.concatenate((dove20, encoded),axis=-1)
print(dove20_with_orb.shape)

(2000, 2000, 4)
(2000, 2000, 9)


In [6]:
# Check for shape symetry and pad with zeros as needed
model_images = [s210m_80, s220m_160, dove20_with_orb, s220m_20]

for num, img in enumerate(model_images):   
    print('Checking image ', num+1)    
    if img.shape[0] != img.shape[1]:
        if img.shape[0] - img.shape[1] <= 3 and img.shape[0] - img.shape[1] > 0:
            factor = img.shape[0] - img.shape[1]
            print('%s more pixels in height' %factor)
            model_images[num] = np.pad(img, ((0, 0), (0, factor), (0, 0)))
            print('Padded with zeros. New shape: ', model_images[num].shape)
        elif img.shape[1] - img.shape[0] <= 3 and img.shape[1] - img.shape[0] > 0:
            factor = img.shape[1] - img.shape[0]        
            print('%s more pixels in width' %factor)
            model_images[num] = np.pad(img, ((0, factor), (0, 0), (0, 0)))
            print('Padded with zeros. New shape: ', model_images[num].shape)
    else:
        print('Shape is good: ', img.shape)

Checking image  1
Shape is good:  (500, 500, 5)
Checking image  2
Shape is good:  (250, 250, 7)
Checking image  3
Shape is good:  (2000, 2000, 9)
Checking image  4
Shape is good:  (2001, 2001, 6)


In [7]:
# Check for compatibility among images
mod_res_compare_size = model_images[0].shape[0]*4

if model_images[2].shape[0] < mod_res_compare_size:
    print('Fixing size compatibility. ')
    diff = mod_res_compare_size - model_images[2].shape[0]
    if diff < 5:
        model_images[2] = np.pad(model_images[2], ((0, 0), (0, diff), (0, 0)))
        model_images[2] = np.pad(model_images[2], ((0, diff), (0, 0), (0, 0)))
        print('Padded with zeros. New shape: ', model_images[2].shape)
    else:
        print('Image size compatibility error.')

In [8]:
def get_test_patches(dset_10m, dset_20m, ps, dset_20gt, patchSize=24, border=4, ps_patch=96, ps_border=16):

    PATCH_SIZE_HR = (patchSize, patchSize)
    PATCH_SIZE_LR = [p//2 for p in PATCH_SIZE_HR]
    BORDER_HR = border
    BORDER_LR = BORDER_HR//2
    PATCH_SIZE_PS = (ps_patch, ps_patch)
    PATCH_SIZE_GT = (ps_patch-2*ps_border, ps_patch-2*ps_border)
    
    # Mirror the data at the borders to have the same dimensions as the input
    dset_10 = np.pad(dset_10m, ((BORDER_HR, BORDER_HR), (BORDER_HR, BORDER_HR), (0, 0)))
    dset_20 = np.pad(dset_20m, ((BORDER_LR, BORDER_LR), (BORDER_LR, BORDER_LR), (0, 0)))
    dset_ps = np.pad(ps, ((ps_border, ps_border), (ps_border, ps_border), (0, 0)))
    dset_gt = np.pad(dset_20gt, ((ps_border, ps_border), (ps_border, ps_border), (0, 0)))  

    BANDS10 = dset_10.shape[2]
    BANDS20 = dset_20.shape[2]
    BANDSps = dset_ps.shape[2]
    BANDSgt = dset_gt.shape[2]
    patchesAlongi = (dset_20.shape[0] - 2 * BORDER_LR) // (PATCH_SIZE_LR[0] - 2 * BORDER_LR)
    patchesAlongj = (dset_20.shape[1] - 2 * BORDER_LR) // (PATCH_SIZE_LR[1] - 2 * BORDER_LR)

    nr_patches = (patchesAlongi + 1) * (patchesAlongj + 1)

    label_20 = np.zeros((nr_patches, BANDSgt) + PATCH_SIZE_GT).astype(np.float32)   #initialize with PATCH_SIZE_PS but crop to PATCH_SIZE_GT
    image_20 = np.zeros((nr_patches, BANDS20) + tuple(PATCH_SIZE_LR)).astype(np.float32)
    image_10 = np.zeros((nr_patches, BANDS10) + PATCH_SIZE_HR).astype(np.float32)
    image_ps = np.zeros((nr_patches, BANDSps) + PATCH_SIZE_PS).astype(np.float32)

    range_i = np.arange(0, (dset_20.shape[0] - 2 * BORDER_LR) // (PATCH_SIZE_LR[0] - 2 * BORDER_LR)) * (
        PATCH_SIZE_LR[0] - 2 * BORDER_LR)
    range_j = np.arange(0, (dset_20.shape[1] - 2 * BORDER_LR) // (PATCH_SIZE_LR[1] - 2 * BORDER_LR)) * (
        PATCH_SIZE_LR[1] - 2 * BORDER_LR)

    if not (np.mod(dset_20.shape[0] - 2 * BORDER_LR, PATCH_SIZE_LR[0] - 2 * BORDER_LR) == 0):
        range_i = np.append(range_i, (dset_20.shape[0] - PATCH_SIZE_LR[0]))
    if not (np.mod(dset_20.shape[1] - 2 * BORDER_LR, PATCH_SIZE_LR[1] - 2 * BORDER_LR) == 0):
        range_j = np.append(range_j, (dset_20.shape[1] - PATCH_SIZE_LR[1]))

    pCount = 0
    for ii in range_i.astype(int):
        for jj in range_j.astype(int):
            upper_left_i = ii
            upper_left_j = jj
            crop_point_lr = [upper_left_i,
                             upper_left_j,
                             upper_left_i + PATCH_SIZE_LR[0],
                             upper_left_j + PATCH_SIZE_LR[1]]
            crop_point_hr = [p*2 for p in crop_point_lr]
            crop_point_ps = [p*4 for p in crop_point_hr]
            crop_point_gt = [p*4 for p in crop_point_hr]

            
            label_20[pCount] = np.rollaxis(dset_gt[crop_point_gt[0]+ps_border:crop_point_gt[2]-ps_border, 
                                                crop_point_gt[1]+ps_border:crop_point_gt[3]-ps_border], 2)
            
            image_20[pCount] = np.rollaxis(dset_20[crop_point_lr[0]:crop_point_lr[2],
                             crop_point_lr[1]:crop_point_lr[3]], 2)
            
            image_10[pCount] = np.rollaxis(dset_10[crop_point_hr[0]:crop_point_hr[2],
                             crop_point_hr[1]:crop_point_hr[3]], 2)
            
            image_ps[pCount] = np.rollaxis(dset_ps[crop_point_ps[0]:crop_point_ps[2],
                             crop_point_ps[1]:crop_point_ps[3]], 2)
            pCount += 1

    return image_10, image_20, image_ps, label_20

In [9]:
images_210, images_220, images_ps, label_20 = get_test_patches(model_images[0], model_images[1], model_images[2], model_images[3])
#images_210, images_220, images_ps, label_20 = get_test_patches(s210m_40, s220m_80, dove10_with_orb, s220m_20)

In [10]:
images_210 = np.moveaxis(images_210, 1, 3)
images_220 = np.moveaxis(images_220, 1, 3)
images_ps = np.moveaxis(images_ps, 1, 3)
label_20 = np.moveaxis(label_20, 1, 3)

print(images_210.shape)
print(images_220.shape)
print(images_ps.shape)
print(label_20.shape)

(1024, 24, 24, 5)
(1024, 12, 12, 7)
(1024, 96, 96, 9)
(1024, 64, 64, 6)


In [11]:
images_210_tr1, images_210_test, images_220_tr1, images_220_test, images_ps_tr1, images_ps_test, label_20_tr1, label_20_test = train_test_split(images_210, images_220, images_ps, label_20, test_size=0.1, random_state=1)
images_210_train, images_210_val, images_220_train, images_220_val, images_ps_train, images_ps_val, label_20_train, label_20_val = train_test_split(images_210_tr1, images_220_tr1, images_ps_tr1, label_20_tr1, test_size=0.1, random_state=1)

In [12]:
lr = 0.0001
batch_size1 = 32
epochs1 = 100
steps_per_epoch = 300
tryout = 200
gpu = 16 #check
sample = 32
mlt = 5
scale = 8
# patch_size = int(scale * 1024)
patch_size = 96

test_only = False

chk = 1
CHANNEL = 3

def shLoss(y_true, y_pred, delta=2.0):
    diff = y_true-y_pred
    dsq = tf.keras.backend.square(delta)
    return tf.keras.backend.mean( dsq * (tf.sqrt(1+ tf.square(diff)/dsq)-1), axis=-1)


def mae(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred), axis=-1)


def PSNRLoss(y_true, y_pred):
        return 10* k.log(255**2 /(k.mean(k.square(y_pred - y_true))))


class SRResnet:
    def L1_loss(self , y_true , y_pred):
        return k.mean(k.abs(y_true - y_pred))
    
    def RDBlocks(self, x, name, count = 6, filter_count=32, RDB_feat=64):
        ## 6 layers of RDB block
        ## this thing need to be in a damn loop for more customisability
        li = [x]
        pas = Convolution2D(filters=filter_count, kernel_size=(3,3), strides=(1, 1), padding='same' , activation='relu' , name = name+'_conv1')(x)

        for i in range(2 , count+1):
            li.append(pas)
            out =  Concatenate(axis = -1)(li) # conctenated output self.channel_axis
            pas = Convolution2D(filters=filter_count, kernel_size=(3,3), strides=(1, 1), padding='same' , activation='relu', name = name+'_conv'+str(i))(out)

        li.append(pas)
        out1 = Concatenate(axis = -1)(li) #self.channel_axis
        feat = Convolution2D(filters=RDB_feat, kernel_size=(1,1), strides=(1, 1), padding='same',activation='relu' , name = name+'_Local_Conv')(out1)

        feat = Add()([feat , x])
        print("RDBlocks",feat)
        return feat
        
    def visualize(self):
        plot_model(self.model, to_file='model.png' , show_shapes = True)
    
    def get_model(self):
        return self.model
    
    def Block_Of_RDBBlocks(self, inp, RDB_count=20, count=6, filter_count=32, RDB_feat=64, end_feat=64):
        
        pass1 = Convolution2D(filters=RDB_feat, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu')(inp)

        pass2 = Convolution2D(filters=RDB_feat, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu')(pass1)


        RDB = self.RDBlocks(pass2 , 'RDB1', count=count, filter_count=filter_count, RDB_feat=RDB_feat)
        RDBlocks_list = [RDB,]
        for i in range(2,RDB_count+1):
            RDB = self.RDBlocks(RDB ,'RDB'+str(i), count=count, filter_count=filter_count, RDB_feat=RDB_feat)
            RDBlocks_list.append(RDB)
        out = Concatenate(axis = -1)(RDBlocks_list) #self.channel_axis
        out = Convolution2D(filters=RDB_feat, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(out)
        output = Add()([out, pass1])
        output = Convolution2D(filters=end_feat, kernel_size=(3,3), strides=(1,1), padding='same', name="rdb_out")(output)
        
        return output


    def __init__(self, s10img, s20img, psimg, lr=0.00005, patch_size=32, RDB_count=4, count=2, filter_count=64, RDB_feat=128, end_feat=128, chk = -1, scale = 4):
        self.channel_axis = 3
        inp10 = Input(shape = (s10img.shape[1], s10img.shape[2], s10img.shape[3]))   # (24,24,4)
        inp20 = Input(shape = (s20img.shape[1], s20img.shape[2], s20img.shape[3]))   # (12,12,6)
        inpPS = Input(shape = (psimg.shape[1], psimg.shape[2], psimg.shape[3]))   # (96,96,9)

        Subpixel_scale8 = Lambda(lambda x:tf.nn.depth_to_space(x,8))
        Subpixel_scale4 = Lambda(lambda x:tf.nn.depth_to_space(x,4))
        
        s220c = Convolution2D(filters = (s20img.shape[3])*8*8*mlt, kernel_size=1, strides=1, padding='valid')(inp20)
        print("S220c shape is", s220c.shape)
        s220s = Subpixel_scale8(inputs=s220c)
        print("S220s shape is", s220s.shape)
        m20 = Model(inputs=inp20, outputs=s220s)
        s210c = Convolution2D(filters = (s10img.shape[3])*4*4*mlt, kernel_size=1, strides=1, padding='valid')(inp10)
        print("S210c shape is", s210c.shape)
        s210s = Subpixel_scale4(inputs=s210c)
        print("S210s shape is", s210s.shape)
        m10 = Model(inputs=inp10, outputs=s210s)
        
        all_inp = Concatenate(axis=-1)([m10.output, m20.output, inpPS])
        print("all_inp shape is", all_inp.shape)
        allb = self.Block_Of_RDBBlocks(all_inp, RDB_count, count, filter_count, RDB_feat, end_feat)
        print("BofRDBlocks",allb)
        allm = Cropping2D(cropping=16)(allb)
        print("Cropping",allm)
        allrc = Convolution2D(filters = 6, kernel_size = 1, strides = 1, padding = "valid", activation = None)(allm)
        
        print("Output", allrc)
        model = Model(inputs=[m10.input, m20.input, inpPS], outputs = allrc)
#         print([n.input_tensors.name for n in model.get_layer('A_3').inbound_nodes])
        adam = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, decay=0, amsgrad=False)

        model.compile(loss=shLoss, optimizer='adam' , metrics=['mae'])

        if chk >=0 :
            print("loading existing weights !!!")
            model.load_weights('final.h5')
        self.model = model

            
    def fit(self, x, y, batch_size, epochs, verbose, callbacks, validation_data, steps_per_epoch):   
        hist = self.model.fit(x, y, batch_size, epochs, verbose, callbacks, validation_data=validation_data, steps_per_epoch=steps_per_epoch)
        return hist.history


chk = -1
net = SRResnet(images_210, images_220, images_ps, lr = lr ,scale = scale , chk = chk)
net.visualize()
# net.get_model().summary()

my_callbacks =[
            tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}',
              monitor = "loss",
              verbose = 1,
              save_best_only = True,
              save_freq = "epoch"
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
              monitor = "loss",
              factor = 0.9,
              patience = 20,
              verbose = 1,
              min_lr = 0.0001 / 10
            ),
            tf.keras.callbacks.EarlyStopping(
              monitor = "loss",
              min_delta = 2,
              patience = 200,
              verbose = 1,
              baseline = None,
              restore_best_weights = False
            ),
            tf.keras.callbacks.TerminateOnNaN(),
            tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]


2021-12-14 12:47:05.284847: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-14 12:47:05.288930: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-14 12:47:05.383062: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 12:47:05.384144: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2021-12-14 12:47:05.384217: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-12-14 12:47:05.430173: I tensorflow/stream_executor/platform/def

S220c shape is (None, 12, 12, 2240)
S220s shape is (None, 96, 96, 35)
S210c shape is (None, 24, 24, 400)
S210s shape is (None, 96, 96, 25)
all_inp shape is (None, 96, 96, 69)
RDBlocks KerasTensor(type_spec=TensorSpec(shape=(None, 96, 96, 128), dtype=tf.float32, name=None), name='add/add:0', description="created by layer 'add'")
RDBlocks KerasTensor(type_spec=TensorSpec(shape=(None, 96, 96, 128), dtype=tf.float32, name=None), name='add_1/add:0', description="created by layer 'add_1'")
RDBlocks KerasTensor(type_spec=TensorSpec(shape=(None, 96, 96, 128), dtype=tf.float32, name=None), name='add_2/add:0', description="created by layer 'add_2'")
RDBlocks KerasTensor(type_spec=TensorSpec(shape=(None, 96, 96, 128), dtype=tf.float32, name=None), name='add_3/add:0', description="created by layer 'add_3'")
BofRDBlocks KerasTensor(type_spec=TensorSpec(shape=(None, 96, 96, 128), dtype=tf.float32, name=None), name='rdb_out/BiasAdd:0', description="created by layer 'rdb_out'")
Cropping KerasTensor(ty

2021-12-14 12:47:06.569173: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-12-14 12:47:06.569194: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-12-14 12:47:06.569894: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1365] Profiler found 1 GPUs
2021-12-14 12:47:06.575333: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcupti.so.10.1
2021-12-14 12:47:06.654278: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-12-14 12:47:06.654372: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1487] CUPTI activity buffer flushed


In [13]:
### TRAIN MODEL ###
model20 = net.get_model()
model20.summary()
%tensorboard --logdir logs/
model20.fit(x=[images_210_train, images_220_train, images_ps_train], y=label_20_train, batch_size=batch_size1, epochs=epochs1, verbose=1, callbacks=my_callbacks, validation_data=([images_210_val, images_220_val, images_ps_val], label_20_val))#, validation_steps = 3) #steps_per_epoch=steps_per_epoch

model20.save(output_folder + date + 'trained_8x_model')

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 24, 24, 5)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 12, 12, 7)]  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 24, 24, 400)  2400        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 12, 12, 2240) 17920       input_2[0][0]                    
____________________________________________________________________________________________

2021-12-14 12:47:08.967714: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-14 12:47:08.987841: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3600000000 Hz


Epoch 1/100


2021-12-14 12:47:09.787240: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-12-14 12:47:10.868696: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2021-12-14 12:47:10.908690: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-12-14 12:47:11.286059: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-12-14 12:47:16.379627: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.55GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-12-14 12:4

 1/26 [>.............................] - ETA: 6:30 - loss: 39933.9727 - mae: 19968.9863

...

26/26 [==============================] - 47s 1s/step - loss: 65655.1110 - mae: 32829.5539 - val_loss: 4212.0898 - val_mae: 2108.0386

Epoch 00001: loss improved from inf to 29803.46094, saving model to model.01-4212.09


2021-12-14 12:47:56.147095: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model.01-4212.09/assets
Epoch 2/100
26/26 [==============================] - 11s 423ms/step - loss: 3873.5924 - mae: 1938.7894 - val_loss: 2866.4133 - val_mae: 1435.1992

Epoch 00002: loss improved from 29803.46094 to 3486.73096, saving model to model.02-2866.41
INFO:tensorflow:Assets written to: model.02-2866.41/assets
Epoch 3/100
26/26 [==============================] - 11s 414ms/step - loss: 2668.4204 - mae: 1336.2005 - val_loss: 1917.1438 - val_mae: 960.5587

Epoch 00003: loss improved from 3486.73096 to 2357.76440, saving model to model.03-1917.14
INFO:tensorflow:Assets written to: model.03-1917.14/assets
Epoch 4/100
26/26 [==============================] - 11s 441ms/step - loss: 1952.9232 - mae: 978.4485 - val_loss: 2577.5681 - val_mae: 1290.7738

Epoch 00004: loss improved from 2357.76440 to 1996.77893, saving model to model.04-2577.57
INFO:tensorflow:Assets written to: model.04-2577.57/assets
Epoch 5/100
26/26 [==============================] 

INFO:tensorflow:Assets written to: model.34-1085.46/assets
Epoch 35/100
26/26 [==============================] - 12s 443ms/step - loss: 1220.6701 - mae: 612.3168 - val_loss: 1195.8362 - val_mae: 599.9020

Epoch 00035: loss did not improve from 1198.85315
Epoch 36/100
26/26 [==============================] - 11s 434ms/step - loss: 1204.3485 - mae: 604.1563 - val_loss: 1430.6982 - val_mae: 717.3375

Epoch 00036: loss did not improve from 1198.85315
Epoch 37/100
26/26 [==============================] - 11s 439ms/step - loss: 1289.2318 - mae: 646.5993 - val_loss: 1388.5420 - val_mae: 696.2578

Epoch 00037: loss did not improve from 1198.85315
Epoch 38/100
26/26 [==============================] - 11s 434ms/step - loss: 1245.4385 - mae: 624.7016 - val_loss: 1223.3555 - val_mae: 613.6601

Epoch 00038: loss improved from 1198.85315 to 1176.28198, saving model to model.38-1223.36
INFO:tensorflow:Assets written to: model.38-1223.36/assets
Epoch 39/100
26/26 [==============================] - 12s


Epoch 00070: loss did not improve from 998.52673
Epoch 71/100
26/26 [==============================] - 11s 435ms/step - loss: 931.6142 - mae: 467.7844 - val_loss: 1001.1781 - val_mae: 502.5676

Epoch 00071: loss improved from 998.52673 to 972.82727, saving model to model.71-1001.18
INFO:tensorflow:Assets written to: model.71-1001.18/assets
Epoch 72/100
26/26 [==============================] - 11s 433ms/step - loss: 1017.3317 - mae: 510.6454 - val_loss: 1100.4969 - val_mae: 552.2317

Epoch 00072: loss did not improve from 972.82727
Epoch 73/100
26/26 [==============================] - 11s 434ms/step - loss: 1019.5060 - mae: 511.7329 - val_loss: 863.1478 - val_mae: 433.5490

Epoch 00073: loss did not improve from 972.82727
Epoch 74/100
26/26 [==============================] - 11s 433ms/step - loss: 992.0514 - mae: 498.0049 - val_loss: 843.6072 - val_mae: 423.7777

Epoch 00074: loss did not improve from 972.82727
Epoch 75/100
26/26 [==============================] - 11s 434ms/step - loss

In [14]:
# Remove old models and log files
files_list = os.listdir()

for file in files_list:
    if 'model' in file:
        shutil.rmtree(file)
        
for file in files_list:
    if 'logs' in file:
        shutil.rmtree(file)